# Krippendorff’s Alpha

## What is Krippendorff’s Alpha Coefficient?

Krippendorff’s alpha (α) is a statistical measure of inter-rater agreement. It assesses how consistently different raters evaluate the same items, accounting for the disagreement that could occur by chance. It is defined by the formula:

$$
\alpha = 1 - \frac{D_o}{D_e}
$$

Where:
- **Dₒ** is the *observed disagreement* (based on the actual data).
- **Dₑ** is the *expected disagreement* (the disagreement expected by chance).

## Interpretation

- **Perfect agreement**  
  If there is no observed disagreement ($D_o = 0$):
  
  $$
  \alpha = 1 - \frac{0}{D_e} = 1
  $$

- **Agreement no better than chance**  
  If the observed disagreement equals the expected disagreement by chance ($D_o = D_e$):
  
  $$
  \alpha = 1 - \frac{D_e}{D_e} = 1 - 1 = 0
  $$

- **Systematic disagreement (worse than chance)**  
  If the observed disagreement exceeds the expected disagreement by chance ($D_o > D_e$):
  
  $$
  \alpha = 1 - \frac{D_o}{D_e} < 0
  $$

## Steps for computing Krippendorff's Alpha

1. Build **reliability data matrix** (raw annotations)

2. Create a **values-by-units matrix** (how many times each value was assigned to each unit).

3. Drop any unit of analysis that was **annotated only once** (there is no possile agreement comparison).

4. Compute:
    - total annotations per value and per unit.
    - the metric difference matrix (ordinal, nominal, interval...).
    - observed disagreement
    - expected disagreement
    - alpha coefficient

### References
[Computing Krippendorff’s Alpha-Reliability](https://www.asc.upenn.edu/sites/default/files/2021-03/Computing%20Krippendorff%27s%20Alpha-Reliability.pdf)

[Wikipedia](https://en.wikipedia.org/wiki/Krippendorff%27s_alpha)

In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install krippendorff
!{sys.executable} -m pip install tabulate

In [2]:
import numpy as np
import pandas as pd
import string
import krippendorff

## Reliability Data Matrix

In [3]:
# raw annotations

annA = [1,2,3,3,2,1,4,1,2,False,False,False]
annB = [1,2,3,3,2,2,4,1,2,5,False,3] 
annC = [False,3,3,3,2,3,4,2,2,5,1,False]
annD = [1,2,3,3,2,4,4,1,2,5,1,False]

In [4]:
# reliability data matrix

data = [annA, annB, annC, annD]
units = ["text1", "text2", "text3", "text4", "text5", "text6", "text7", "text8", "text9", "text10", "text11", "text12"]
annotators = ["annA", "annB", "annC", "annD"]
dfrel = pd.DataFrame(data, columns=units, index=annotators)
dfrel

,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11,text12
annA,1,2,3,3,2,1,4,1,2,False,False,False
annB,1,2,3,3,2,2,4,1,2,5,False,3
annC,False,3,3,3,2,3,4,2,2,5,1,False
annD,1,2,3,3,2,4,4,1,2,5,1,False


## Values by Units Matrix

In [5]:
# values by units matrix
# how many times each value was assigned to each unit

values = sorted(set([v for value in dfrel.values for v in value if v != False]))
print(f"values: {values}")

data = [[list(dfrel[col].values).count(value) for value in values] for col in dfrel.columns]
dfvu = pd.DataFrame(data)
dfvu = dfvu.T
dfvu.columns = units
dfvu.index = values
dfvu

values: [1, 2, 3, 4, 5]


,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11,text12
1,3,0,0,0,0,1,0,3,0,0,2,0
2,0,3,0,0,4,1,0,1,4,0,0,0
3,0,1,4,4,0,1,0,0,0,0,0,1
4,0,0,0,0,0,1,4,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,3,0,0


## Agreement is measured between at least one pair of annotations

In [6]:
# dropping columns with not pairing units (e.g. text12)
not_paired_units = []
for col in dfvu.columns:
    if dfvu[col].sum() <= 1:
        not_paired_units.append(col)

dfvu = dfvu.drop(col, axis=1)
dfvu

,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11
1,3,0,0,0,0,1,0,3,0,0,2
2,0,3,0,0,4,1,0,1,4,0,0
3,0,1,4,4,0,1,0,0,0,0,0
4,0,0,0,0,0,1,4,0,0,0,0
5,0,0,0,0,0,0,0,0,0,3,0


In [7]:
dfvu["N"] = dfvu.sum(axis=1) # total annotations per value
dfvu.loc[len(dfvu)+1] = dfvu.sum(axis=0) # total annotations per unit

dfvu.rename({len(dfvu):"total_unit"}, inplace=True)
dfvu

,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11,N
1,3,0,0,0,0,1,0,3,0,0,2,9
2,0,3,0,0,4,1,0,1,4,0,0,13
3,0,1,4,4,0,1,0,0,0,0,0,10
4,0,0,0,0,0,1,4,0,0,0,0,5
5,0,0,0,0,0,0,0,0,0,3,0,3
total_unit,3,4,4,4,4,4,4,4,4,3,2,40


## Metric differences: nominal, ordinal, interval

In [8]:
# nominal metric differences
data = []
for c in values:
    data.append([])
    for k in values:
        if c == k:
            data[-1].append(0)
        else:
            data[-1].append(1)
        
diffdf = pd.DataFrame(data, columns=values, index=values)
diffdf

,1,2,3,4,5
1,0,1,1,1,1
2,1,0,1,1,1
3,1,1,0,1,1
4,1,1,1,0,1
5,1,1,1,1,0


In [9]:
# ordinal metric differences
data = []
for c in values:
    data.append([])
    for k in values:
        if c == k:
            data[-1].append(0)
        else:
            n_g = sum([dfvu.loc[i, "N"] for i in range(c, k+1)])
            data[-1].append((n_g - ((dfvu.loc[c, "N"] + dfvu.loc[k, "N"])/2))**2)
        
diffdf = pd.DataFrame(data, columns=values, index=values)
diffdf

,1,2,3,4,5
1,0.00,121.00,506.25,900.00,1156.00
2,121.00,0.00,132.25,361.00,529.00
3,90.25,132.25,0.00,56.25,132.25
4,49.00,81.00,56.25,0.00,16.00
5,36.00,64.00,42.25,16.00,0.00


In [10]:
# interval metric differences (doesn't work with strings)
data = []
for c in values:
    data.append([])
    for k in values:
        if c == k:
            data[-1].append(0)
        else:
            data[-1].append((c-k)**2) # cannot substract strings!
        
diffdf = pd.DataFrame(data, columns=values, index=values)
diffdf

,1,2,3,4,5
1,0,1,4,9,16
2,1,0,1,4,9
3,4,1,0,1,4
4,9,4,1,0,1
5,16,9,4,1,0


In [11]:
def difference(c,k,measure, dfvu):
    if measure == 'nominal':
        return 0 if c == k else 1

    if measure == 'ordinal':
        n_g = sum([dfvu.loc[i, "N"] for i in range(c, k+1)])
        return (n_g - ((dfvu.loc[c, "N"] + dfvu.loc[k, "N"])/2))**2

    if measure == 'interval':
        return (c-k)**2

In [12]:
measure = 'nominal'

## Computing Observed Disagreement

In [13]:
dfvu
# the observed disagreement is the normalized sum of disagrement found between pairs of values per each unit of analysis
# going from text1 up to text11
# if the text was annotated with more than one value, add up their difference to the observed disagreement

,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11,N
1,3,0,0,0,0,1,0,3,0,0,2,9
2,0,3,0,0,4,1,0,1,4,0,0,13
3,0,1,4,4,0,1,0,0,0,0,0,10
4,0,0,0,0,0,1,4,0,0,0,0,5
5,0,0,0,0,0,0,0,0,0,3,0,3
total_unit,3,4,4,4,4,4,4,4,4,3,2,40


In [14]:
D_o = 0
for unit in dfvu.columns.values[:-1]: # for text in [text1, text2, text3...]
    n_unit_dot = dfvu.loc["total_unit", unit] # total values assigned to text
    sum_ck = 0
    for c in range(1, dfvu.shape[0]): # traversing values (1,2,3,4,5)
        n_uc = dfvu.loc[c,unit] # n_uc is number of values c assigned to unit u (how many times value 1 was assigned to text1)
        for k in range(c+1, dfvu.shape[0]):
            n_uk = dfvu.loc[k,unit] # n_uk is number of values k (c+1) assigned to unit u
            delta_ck = difference(c,k,measure,dfvu) # metric difference between values c and k
            sum_ck += n_uc * n_uk * delta_ck
    D_o += (1/(n_unit_dot - 1)) * sum_ck
print('D_o', D_o)  
# normalized sum of the observed disagreement per unit between value pairs --> 1,1; 1,2; 1,3; 1,4; 1,5; 2,3; 2,4; 2,5; 3,4; 3,5; 4,5


D_o 4.0


In [15]:
# observed disagreement for 3 different units: text1, text2, text6

highlight = ['text1', 'text2', 'text6']
D_o = 0
for unit in dfvu.columns.values[:-1]: # for text in [text1, text2, text3...]
    n_unit_dot = dfvu.loc["total_unit", unit] # total values assigned to unit
    sum_ck = 0
    for c in range(1, dfvu.shape[0]): # traversing values (1,2,3,4,5)
        n_uc = dfvu.loc[c,unit] # n_uc is number of values c assigned to unit u (how many times value 1 was assigned to text1)
        for k in range(c+1, dfvu.shape[0]):
            n_uk = dfvu.loc[k,unit] # n_uk is number of values k (c+1) assigned to unit u
            delta_ck = difference(c,k,measure,dfvu) # metric difference between values c and k
            sum_ck += n_uc * n_uk * delta_ck

            # highlighting
            if unit in highlight:
                if n_uc * n_uk * delta_ck != 0:
                    print(f'unit {unit}, value {c} compared to value {k}, observed disagreement: {n_uc * n_uk * delta_ck}')
    
    D_o += (1/(n_unit_dot - 1)) * sum_ck # observed disagreement normalized by total values assigned to unit
print('D_o', D_o)
dfvu

unit text2, value 2 compared to value 3, observed disagreement: 3
unit text6, value 1 compared to value 2, observed disagreement: 1
unit text6, value 1 compared to value 3, observed disagreement: 1
unit text6, value 1 compared to value 4, observed disagreement: 1
unit text6, value 2 compared to value 3, observed disagreement: 1
unit text6, value 2 compared to value 4, observed disagreement: 1
unit text6, value 3 compared to value 4, observed disagreement: 1
D_o 4.0


,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11,N
1,3,0,0,0,0,1,0,3,0,0,2,9
2,0,3,0,0,4,1,0,1,4,0,0,13
3,0,1,4,4,0,1,0,0,0,0,0,10
4,0,0,0,0,0,1,4,0,0,0,0,5
5,0,0,0,0,0,0,0,0,0,3,0,3
total_unit,3,4,4,4,4,4,4,4,4,3,2,40


## Computing Expected Disagreement

In [16]:
# expected disagreement is the sum of the disagreement we would expect by chance 
# if annotations were assigned randomly but respecting the overall distribution between a pair of values

In [17]:
n_dotdot = dfvu.loc["total_unit", "N"]
print('Total number of annotations:', n_dotdot)

D_e = 0
for c in range(1, dfvu.shape[0]): # traversing values (1,2,3,4,5)
    n_c = dfvu.loc[c, "N"] # total number of times that value c was assigned to all units
    for k in range(c+1, dfvu.shape[0]):
        n_k = dfvu.loc[k, "N"] # total number of times that value k (c+1) was assigned to all units
        D_e += n_c * n_k * difference(c,k,measure,dfvu)
print('D_e', D_e)

Total number of annotations: 40
D_e 608


In [18]:
# expected disagreement for value 1

highlight = [1]
D_e = 0
for c in range(1, dfvu.shape[0]): # traversing values (1,2,3,4,5)
    n_c = dfvu.loc[c, "N"] # total number of times that value c was assigned to any unit
    for k in range(c+1, dfvu.shape[0]):
        n_k = dfvu.loc[k, "N"] # total number of times that value k was assigned to any unit
        D_e += n_c * n_k * difference(c,k,measure,dfvu)
        if c in highlight:
            print(f'value {c} was assigned {n_c} times, value {k} was assigned {n_k}, their difference is {difference(c,k,measure,dfvu)}, and multiplied sum up to {n_c * n_k * difference(c,k,measure,dfvu)}')
dfvu

# note that if a value was not assigned to any unit, then it will sum up 0 to the overall expected disagreement!


value 1 was assigned 9 times, value 2 was assigned 13, their difference is 1, and multiplied sum up to 117
value 1 was assigned 9 times, value 3 was assigned 10, their difference is 1, and multiplied sum up to 90
value 1 was assigned 9 times, value 4 was assigned 5, their difference is 1, and multiplied sum up to 45
value 1 was assigned 9 times, value 5 was assigned 3, their difference is 1, and multiplied sum up to 27


,text1,text2,text3,text4,text5,text6,text7,text8,text9,text10,text11,N
1,3,0,0,0,0,1,0,3,0,0,2,9
2,0,3,0,0,4,1,0,1,4,0,0,13
3,0,1,4,4,0,1,0,0,0,0,0,10
4,0,0,0,0,0,1,4,0,0,0,0,5
5,0,0,0,0,0,0,0,0,0,3,0,3
total_unit,3,4,4,4,4,4,4,4,4,3,2,40


## Computing alpha coefficient

In [19]:
alpha = 1 - ((n_dotdot-1)*(D_o/D_e))
alpha

np.float64(0.743421052631579)

## All in one go

In [20]:
def difference(c,k,measure, dfvu):
    if measure == 'nominal':
        return 0 if c == k else 1

    if measure == 'ordinal':
        c = dfvu.index.get_loc(c)
        k = dfvu.index.get_loc(k)
        n_g = sum([dfvu.loc[dfvu.index.values[i], "N"] for i in range(c, k+1)])
        return (n_g - ((dfvu.loc[dfvu.index.values[c], "N"] + dfvu.loc[dfvu.index.values[k], "N"])/2))**2

    if measure == 'interval': # strings only!
        return (c-k)**2
        
def compute_alpha(annotations, measure, value_domain=None):
    # reliability data matrix
    units = [f"text{i+1}" for i in range(len(annotations[0]))]
    annotators = [f"ann{string.ascii_uppercase[i]}" for i in range(len(annotations))]
    dfrel = pd.DataFrame(annotations, columns=units, index=annotators)

    # values by units matrix
    values = set([v for row in dfrel.values for v in row if pd.notna(v)])
    if measure == 'nominal':
        values = sorted(values)
    else:
        values = value_domain # ordinal and interval require a predefined order
        
    data = [[list(dfrel[col].values).count(value) for value in values] for col in dfrel.columns]
    dfvu = pd.DataFrame(data)
    dfvu = dfvu.T
    dfvu.columns = units
    dfvu.index = values
    
    # dropping columns with not pairing units (e.g. text12)
    not_paired_units = []
    for col in dfvu.columns:
        if dfvu[col].sum() <= 1:
            not_paired_units.append(col)
    dfvu = dfvu.drop(not_paired_units, axis=1)
    
    # total annotations
    dfvu["N"] = dfvu.sum(axis=1) # total annotations per value
    dfvu.loc[len(dfvu)+1] = dfvu.sum(axis=0) # total annotations per unit
    dfvu.rename({len(dfvu):"total_unit"}, inplace=True)
        
    D_o = 0
    n_dotdot = 0
    for unit in dfvu.columns.values[:-1]:
        n_unit_dot = dfvu.loc["total_unit", unit]

        n_dotdot += n_unit_dot
        
        sum_ck = 0
        for i in range(dfvu.shape[0]-1):
            c = dfvu.index[i]
            n_uc = dfvu.loc[c,unit]
            for j in range(i+1, dfvu.shape[0]-1):
                k = dfvu.index[j]
                n_uk = dfvu.loc[k,unit]
                delta_ck = difference(c,k,measure,dfvu)
                sum_ck += n_uc * n_uk * delta_ck
        D_o += (1/(n_unit_dot - 1)) * sum_ck
    
    D_e = 0
    for i in range(dfvu.shape[0]):
        c = dfvu.index[i]
        n_c = dfvu.loc[c, "N"]
        for j in range(i+1, dfvu.shape[0]-1):
            k = dfvu.index[j]
            n_k = dfvu.loc[k, "N"]
            
            D_e += n_c * n_k * difference(c,k,measure,dfvu)
    
    alpha = 1 - ((n_dotdot-1)*(D_o/D_e))
    return alpha, dfrel, dfvu, D_o, D_e

In [21]:
annA = [1,2,3,3,2,1,4,1,2,np.nan,np.nan,np.nan]
annB = [1,2,3,3,2,2,4,1,2,5,np.nan,3] 
annC = [np.nan,3,3,3,2,3,4,2,2,5,1,np.nan]
annD = [1,2,3,3,2,4,4,1,2,5,1,np.nan]
data = [annA, annB, annC, annD]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'nominal')
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement="nominal"))
print()
#print(dfrel.to_markdown())
print()
#print(dfvu.to_markdown())

alpha 0.743421052631579
D_o 4.0
D_e 608
krippendorff module 0.743421052631579




In [40]:
annA = [1,1,1,1,1,1,1,2,2,2,2]
annB = [1,1,1,1,1,1,2,2,2,2,2]
data = [annA, annB]
print('total samples', len(annA))

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'nominal')
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement="nominal"))
print()
#print(dfrel.to_markdown())
print()
#print(dfvu.to_markdown())

total samples 11
alpha 0.8205128205128205
D_o 1.0
D_e 117
krippendorff module 0.8205128205128205




In [39]:
annA = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2]
annB = [1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2]
data = [annA, annB]
print('total samples', len(annA))
alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'nominal')
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement="nominal"))
print()
#print(dfrel.to_markdown())
print()
#print(dfvu.to_markdown())

total samples 18
alpha 0.8559670781893004
D_o 1.0
D_e 243
krippendorff module 0.8559670781893004




In [41]:
annA = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2]
annB = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2]
data = [annA, annB]
print('total samples', len(annA))
alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'nominal')
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement="nominal"))
print()
#print(dfrel.to_markdown())
print()
#print(dfvu.to_markdown())

total samples 31
alpha 0.8721174004192872
D_o 1.0
D_e 477
krippendorff module 0.8721174004192872




In [42]:
annA = [3,3,3,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2]
annB = [3,3,3,3,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2]
data = [annA, annB]
print('total samples', len(annA))
alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'nominal')
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement="nominal"))
print()
#print(dfrel.to_markdown())
print()
#print(dfvu.to_markdown())

total samples 31
alpha 0.9327453142227122
D_o 1.0
D_e 907
krippendorff module 0.9327453142227122




In [23]:
data = [
    [np.nan, np.nan, np.nan, np.nan, np.nan, 3, 4, 1, 2, 1, 1, 3, 3, np.nan, 3],
    [1, np.nan, 2, 1, 3, 3, 4, 3, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    [np.nan, np.nan, 2, 1, 3, 4, 4, np.nan, 2, 1, 1, 3, 3, np.nan, 4]
]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'nominal')
print("alpha", alpha)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='nominal'))
print()

alpha 0.691358024691358
krippendorff module 0.691358024691358



In [24]:
data = [
    [np.nan, np.nan, np.nan, np.nan, np.nan, 3, 4, 1, 2, 1, 1, 3, 3, np.nan, 3],
    [1, np.nan, 2, 1, 3, 3, 4, 3, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    [np.nan, np.nan, 2, 1, 3, 4, 4, np.nan, 2, 1, 1, 3, 3, np.nan, 4]
]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'ordinal', value_domain=[1,2,3,4])
print("alpha", alpha)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='ordinal', value_domain=[1,2,3,4]))
print()

alpha 0.8067214199413153
krippendorff module 0.8067214199413153



In [25]:
annA = ["low", "low", "high"]
annB = ["high", "low", "high"]
data = [annA, annB]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'nominal')
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='nominal'))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha 0.4444444444444444
D_o 1.0
D_e 9
krippendorff module 0.4444444444444444

|      | text1   | text2   | text3   |
|:-----|:--------|:--------|:--------|
| annA | low     | low     | high    |
| annB | high    | low     | high    |

|            |   text1 |   text2 |   text3 |   N |
|:-----------|--------:|--------:|--------:|----:|
| high       |       1 |       0 |       2 |   3 |
| low        |       1 |       2 |       0 |   3 |
| total_unit |       2 |       2 |       2 |   6 |


In [26]:
annA = ["low", "low", "high"]
annB = ["high", "low", "high"]
data = [annA, annB]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'ordinal', value_domain=['low', 'high'])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='ordinal', value_domain=['low', 'high']))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha 0.4444444444444444
D_o 9.0
D_e 81.0
krippendorff module 0.4444444444444444

|      | text1   | text2   | text3   |
|:-----|:--------|:--------|:--------|
| annA | low     | low     | high    |
| annB | high    | low     | high    |

|            |   text1 |   text2 |   text3 |   N |
|:-----------|--------:|--------:|--------:|----:|
| low        |       1 |       2 |       0 |   3 |
| high       |       1 |       0 |       2 |   3 |
| total_unit |       2 |       2 |       2 |   6 |


In [27]:
annA = ["low", "low", "low", "low", "high"]
annB = ["high", "low", "high", "low", "high"]
data = [annA, annB]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'ordinal', value_domain=['low', 'high'])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='ordinal', value_domain=['low', 'high']))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha 0.25
D_o 50.0
D_e 600.0
krippendorff module 0.2499999999999999

|      | text1   | text2   | text3   | text4   | text5   |
|:-----|:--------|:--------|:--------|:--------|:--------|
| annA | low     | low     | low     | low     | high    |
| annB | high    | low     | high    | low     | high    |

|            |   text1 |   text2 |   text3 |   text4 |   text5 |   N |
|:-----------|--------:|--------:|--------:|--------:|--------:|----:|
| low        |       1 |       2 |       1 |       2 |       0 |   6 |
| high       |       1 |       0 |       1 |       0 |       2 |   4 |
| total_unit |       2 |       2 |       2 |       2 |       2 |  10 |


In [28]:
annA = [1,1,1,2,2,4,4]
annB = [1,1,3,3,4,4,4]
data = [annA, annB]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'interval', value_domain=[1,2,3,4])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='interval', value_domain=[1,2,3,4]))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha 0.6443768996960486
D_o 9.0
D_e 329
krippendorff module 0.6443768996960486

|      |   text1 |   text2 |   text3 |   text4 |   text5 |   text6 |   text7 |
|:-----|--------:|--------:|--------:|--------:|--------:|--------:|--------:|
| annA |       1 |       1 |       1 |       2 |       2 |       4 |       4 |
| annB |       1 |       1 |       3 |       3 |       4 |       4 |       4 |

|            |   text1 |   text2 |   text3 |   text4 |   text5 |   text6 |   text7 |   N |
|:-----------|--------:|--------:|--------:|--------:|--------:|--------:|--------:|----:|
| 1          |       2 |       2 |       1 |       0 |       0 |       0 |       0 |   5 |
| 2          |       0 |       0 |       0 |       1 |       1 |       0 |       0 |   2 |
| 3          |       0 |       0 |       1 |       1 |       0 |       0 |       0 |   2 |
| 4          |       0 |       0 |       0 |       0 |       1 |       2 |       2 |   5 |
| total_unit |       2 |       2 |       2 |       2 |  

In [29]:
annA = [1,1,1,1,1]
annB = [1,1,2,2,2]
data = [annA, annB]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'interval', value_domain=[1,2,3,4])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='interval', value_domain=[1,2,3,4]))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha -0.2857142857142856
D_o 3.0
D_e 21
krippendorff module -0.2857142857142856

|      |   text1 |   text2 |   text3 |   text4 |   text5 |
|:-----|--------:|--------:|--------:|--------:|--------:|
| annA |       1 |       1 |       1 |       1 |       1 |
| annB |       1 |       1 |       2 |       2 |       2 |

|            |   text1 |   text2 |   text3 |   text4 |   text5 |   N |
|:-----------|--------:|--------:|--------:|--------:|--------:|----:|
| 1          |       2 |       2 |       1 |       1 |       1 |   7 |
| 2          |       0 |       0 |       1 |       1 |       1 |   3 |
| 3          |       0 |       0 |       0 |       0 |       0 |   0 |
| 4          |       0 |       0 |       0 |       0 |       0 |   0 |
| total_unit |       2 |       2 |       2 |       2 |       2 |  10 |


In [30]:
annA = [1,1,1,1,1]
annB = [1,1,1,2,2]
data = [annA, annB]

alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'interval', value_domain=[1,2])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='interval', value_domain=[1,2]))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha -0.125
D_o 2.0
D_e 16
krippendorff module -0.125

|      |   text1 |   text2 |   text3 |   text4 |   text5 |
|:-----|--------:|--------:|--------:|--------:|--------:|
| annA |       1 |       1 |       1 |       1 |       1 |
| annB |       1 |       1 |       1 |       2 |       2 |

|            |   text1 |   text2 |   text3 |   text4 |   text5 |   N |
|:-----------|--------:|--------:|--------:|--------:|--------:|----:|
| 1          |       2 |       2 |       2 |       1 |       1 |   8 |
| 2          |       0 |       0 |       0 |       1 |       1 |   2 |
| total_unit |       2 |       2 |       2 |       2 |       2 |  10 |


In [31]:
# Alpha is high when Do (observed disagreement) is low and De (expected disagreement) is high.
# For De, the value pairs with a span of 2 or 3 (such as value pairs 1/3 or 1/4) occur together 
# are weighted highest (or penalized most). In our data, however, 3 is missing entirely, 
# and 4 only occurs twice, so that the large, highly weighted spans do not occur often, 
# which leads to a rather low number in the denominator (Pe). 
# On the other hand, observed disagreement is large, because the only values occuring in 
# the matrix are disagreements.

# Alpha increases if there are more agreements in the annotation data (case 1) or more values that lead to large spans in possible value pairs are used (case 2).
annA = [1,1,2,2]
annB = [2,2,4,4]
data = [annA, annB]
alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'interval', value_domain=[1,2,3,4])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='interval', value_domain=[1,2,3,4]))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha 0.07894736842105265
D_o 10.0
D_e 76
krippendorff module 0.07894736842105254

|      |   text1 |   text2 |   text3 |   text4 |
|:-----|--------:|--------:|--------:|--------:|
| annA |       1 |       1 |       2 |       2 |
| annB |       2 |       2 |       4 |       4 |

|            |   text1 |   text2 |   text3 |   text4 |   N |
|:-----------|--------:|--------:|--------:|--------:|----:|
| 1          |       1 |       1 |       0 |       0 |   2 |
| 2          |       1 |       1 |       1 |       1 |   4 |
| 3          |       0 |       0 |       0 |       0 |   0 |
| 4          |       0 |       0 |       1 |       1 |   2 |
| total_unit |       2 |       2 |       2 |       2 |   8 |


In [32]:
# case 1 (more agreements): 
annA = [1,1,2,2]
annB = [1,2,4,4]
data = [annA, annB]
alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'interval', value_domain=[1,2,3,4])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='interval', value_domain=[1, 2, 3, 4]))
#print()
#print(dfrel.to_markdown())
#print()
#print(dfvu.to_markdown())


alpha 0.27586206896551724
D_o 9.0
D_e 87
krippendorff module 0.27586206896551724


In [33]:
# case 2 (large-span value pairs for Pe):
annA = [1,2,3,4]
annB = [2,3,4,5]
data = [annA, annB]
alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'ordinal', value_domain=[1,2,3,4, 5])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='ordinal', value_domain=[1, 2, 3, 4, 5]))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha 0.7299382716049383
D_o 12.5
D_e 324.0
krippendorff module 0.7299382716049383

|      |   text1 |   text2 |   text3 |   text4 |
|:-----|--------:|--------:|--------:|--------:|
| annA |       1 |       2 |       3 |       4 |
| annB |       2 |       3 |       4 |       5 |

|            |   text1 |   text2 |   text3 |   text4 |   N |
|:-----------|--------:|--------:|--------:|--------:|----:|
| 1          |       1 |       0 |       0 |       0 |   1 |
| 2          |       1 |       1 |       0 |       0 |   2 |
| 3          |       0 |       1 |       1 |       0 |   2 |
| 4          |       0 |       0 |       1 |       1 |   2 |
| 5          |       0 |       0 |       0 |       1 |   1 |
| total_unit |       2 |       2 |       2 |       2 |   8 |


In [34]:
# Related edge case problem: ties (annotators assign similar values to many items)
# Related to above, because there is no agreement, so the numerator Po will always be large.
# At the same time, De will be comparatively small, as not many diverse value types are present.

annA = [4,4,4,4]
annB = [1,1,1,1]
data = [annA, annB]
alpha, dfrel, dfvu, D_o, D_e = compute_alpha(data, 'ordinal', value_domain=[1,2, 3, 4])
print("alpha", alpha)
print("D_o", D_o)
print("D_e", D_e)
print('krippendorff module', krippendorff.alpha(reliability_data=data, level_of_measurement='ordinal', value_domain=[1, 2, 3, 4]))
print()
print(dfrel.to_markdown())
print()
print(dfvu.to_markdown())

alpha -0.75
D_o 64.0
D_e 256.0
krippendorff module -0.75

|      |   text1 |   text2 |   text3 |   text4 |
|:-----|--------:|--------:|--------:|--------:|
| annA |       4 |       4 |       4 |       4 |
| annB |       1 |       1 |       1 |       1 |

|            |   text1 |   text2 |   text3 |   text4 |   N |
|:-----------|--------:|--------:|--------:|--------:|----:|
| 1          |       1 |       1 |       1 |       1 |   4 |
| 2          |       0 |       0 |       0 |       0 |   0 |
| 3          |       0 |       0 |       0 |       0 |   0 |
| 4          |       1 |       1 |       1 |       1 |   4 |
| total_unit |       2 |       2 |       2 |       2 |   8 |


In [35]:
# To conclude: Krippendorffs alpha (ordinal) does not generally cope 
# poorly with shifted annotations, but it does cope poorly if there 
# are (shifted) ties in the annotations.